<a href="https://colab.research.google.com/github/ahmed192a/ImageProcessing-PatternRecognition/blob/main/P2.Vehicle-Detection/P2_Vehicle_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# P5: Vehicle Detection and Tracking

# Imports

In [ ]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import pickle
import cv2
import glob
import time
from random import shuffle
from scipy.ndimage.measurements import label
from skimage.feature import hog
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from moviepy.editor import VideoFileClip

# Useful functions to be used in the image pipeline for Vehicle Detection and Tracking

# Find Car utils

# Find Car

# Training utils

# Main Training

# Image Pipeline utils

# Image Pipeline

# Test output